In [1]:
import os
import sys
import numpy as np
from astropy.io import fits
import glob
import matplotlib.pyplot as plt

In [2]:
sspmodel_dir = '/Users/aprado/Desktop/sspmodel_py/' #'/home/gmosby/sspmodel_py_clean/sspmodel_py/'

# legac_dir in general will be arbitrary
legac_dir =  '/Users/aprado/Desktop/astropy/spectraDR/' #'/home/gmosby/LEGAC/spectraDR/'
#Find input spectrum
specfiles = glob.glob(sspmodel_dir+'example/*_off_axis.fits')
# print("Found %d FITS files"%(len(specfiles)))
# print(specfiles)

In [3]:
spechdu = fits.open(specfiles[0]) # HDU List for the sspmodel fits files
spechdr = spechdu[0].header # Header for the sspmodel fits file

# spechdr for sspmodel fits files: need to have NAXIS = 2 (so the data we save needs to have 2 dimensions)
# So, we need NAXIS1, NAXIS2, also need CRVAL1 and CDELT1 keywords -- to reconstruct the wavelength for each flux point without carrying the wavelength array values
# we'll use Angstroms for units

In [4]:
# CONVERTING LEGA-C SPECTRA into 1D spectra fits files
#Grab the FITS Binary table for the LEGA-C spectra files
legafile = glob.glob(legac_dir+'*.fits')

In [5]:
def convert_legac_spec(legac_file):
    """Converts LEGA-C Spectra FITS files to 1D Spectra fits files. 
    Takes in the name of a LEGA-C spectrum with the full path name"""
    
#     legafiles = glob.glob(legac_dir+'*fits')
    legac_spec_HDU = fits.open(legac_files)
    legac_spec_header = legac_spec_HDU[0]
    
    #Find the WAVELMIN & WAVELMAX info in the legac file's header to get max & min values for wavelength
    wavel_min = legac_spec_header['WAVELMIN']
    wavel_max = legac_spec_header['WAVELMAX']
    spec_bin = legac_spec_header['SPEC_BIN']
#      ['SIGMA STARS PRIME']
    
    
    nwave = (wavel_max - wavel_min)/spec_bin + 1
    
    #Get actual wavelengths to convert from nm --> angstroms
    waves_nm = wavel_min + spec_bin*np.arange(nwave)
    waves_ang = waves_nm * 10
    
    #Use WAVELMIN as the CRVAL1 keyword in the sspmodel / 1D spec files 
    # but first need to convert units to Angstroms (or check if they're already in Angstroms?)
    #Setting header values to the expected header variable names for SSPMODEL
    cdelt = spec_bin * 10
    crval1 = wavel_min * 10
    
    
    #Use SPEC_BIN as the CDELT1 keyword, but also convert (or check if its in) Angstroms
    #CRVAL1 and CDELT1 are in meters but can also use angstroms